### Punkt 1 i 2
Wybór białka (HNRNPC/HNRNPA2B1) analizowanego podczas eksperymentów, co
jednoznacznie identyfikuje wykorzystywany zestaw danych.

Zapoznanie się z motywem oczekiwanym, wyekstrahowanym na podstawie
eksperymentalnie określonej struktury 3D kompleksu, utworzonego przez analizowane
białko wiążące z RNA, w celu identyfikacji liczby wchodzących w jego skład nukleotydów
(len). Załóżmy, że nasz motyw oczekiwany to TTTT, wtedy len = 4.

In [1]:
import numpy as np
import pandas as pd
import stumpy

import math
from collections import Counter, defaultdict
from matplotlib import pyplot as plt
from sklearn.cluster import DBSCAN, OPTICS, KMeans
from sklearn.decomposition import PCA
from src.datasets.dataset import Dataset

In [2]:
dataset = Dataset.read('./resources/datasets/HNRNPA2B1')

[memo::read-406c9d...] Loading from memory.


In [3]:
print(dataset.pattern.shape)
dataset.pattern

(11, 2)


,fshape_coefficient,base
0,0.0,NaN
1,1.0,NaN
2,2.0,A
3,3.0,G
4,2.0,G
5,0.0,NaN
6,0.0,NaN
7,1.0,T
8,0.5,A
9,0.5,G


In [4]:
print(len(dataset.searches))
dataset.searches[0].head()

1098


,fshape_coefficient,base,shape_coefficient
0,NaN,G,NaN
1,NaN,C,NaN
2,0.410644,A,NaN
3,4.847446,C,NaN
4,0.286998,A,NaN


In [5]:
print(len(dataset.sites))
dataset.sites[0].head()

71


,fshape_coefficient,base
0,4.222436,T
1,2.337724,T
2,4.173553,G
3,4.839571,A
4,4.003999,A


### Punkt 3
Określenie długości potencjalnie obiecujących motywów w = {len, len + 1, len + 2}.
Załóżmy, że nasz motyw oczekiwany to TTTT (len = 4), wtedy len + 1 = {NTTTT, TTTTN}, a
len + 2 = {NTTTTN}, gdzie N to dowolny nukleotyd.

In [6]:
expected_motif_length = len(dataset.pattern)
window_sizes = [expected_motif_length + x for x in range(3)]
window_sizes

[11, 12, 13]

In [7]:
def is_motif_promising(window: pd.Series) -> bool:
    return not window.isnull().any() and (window > 1).any()

### Punkt 4

Ekstrakcja wszystkich obiecujących, ciągłych motywów (reprezentowanych przez
odpowiadające im profile danych fSHAPE) o długości w = len … len + 2 spełniających
założoną charakterystykę miejsca wiązania, w oparciu o wartości współczynnika
reaktywności uzyskane metodą fSHAPE (tzn. wartość reaktywności dla przynajmniej
jednego nukleotydu w ramach motywu musi przekraczać wartość 1.0), spośród krótkich
fragmentów transkryptów zawartych w archiwum
identyfikator_wybranego_białka_binding_sites_fshape.zip, które prawdopodobnie
zawierają nieodkryte dotąd miejsca wiązań RNA z rozpatrywanym białkiem.

In [8]:
promising_fshapes = defaultdict(list)
promising_motifs = defaultdict(list)

for site in dataset.sites:
    for window_size in window_sizes:
        for i in range(len(site) - window_size + 1):
            window = site.fshape_coefficient[i : i + window_size]

            if is_motif_promising(window):
                promising_fshapes[window_size].append(window.to_numpy())
                promising_motifs[window_size].append(site.base[i : i + window_size].to_numpy())

print(
    f"Example\nFshape: {promising_fshapes[window_sizes[0]][0]}\nMotif: {promising_motifs[window_sizes[0]][0]}"
)

Example
Fshape: [  4.2224361    2.33772398   4.17355287   4.83957116   4.00399883
   1.51389289   0.51807962 -10.56896471  -7.17275854 -11.53065754
  -3.00704761]
Motif: ['T' 'T' 'G' 'A' 'A' 'T' 'T' 'G' 'A' 'T' 'A']
